
<h1><center>Documnt Ranking</center></h1>
<h2><center>Building an Information Retrieval System</center></h2>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
root_path = 'gdrive/My Drive/Assignment 1.zip'  #change dir to your project folder

In [ ]:
from zipfile import ZipFile 
filename = root_path
with ZipFile(filename,'r') as Zip:  
  Zip.extractall()  
  print('done')

In [ ]:
#Improt the required Libraries
import os

import nltk
from nltk.corpus import stopwords

import string
nltk.download('stopwords')

import numpy as np
from numpy import array
from numpy import count_nonzero
from scipy.sparse import *
from collections import Counter

import pandas as pd


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)

In [ ]:
#Reading the directory of our dataset of documents
files = [] 
for file in os.listdir(os.path.join('Assignment 1/data')):    
  if file.endswith(".txt"):        
    tmp = os.path.join('Assignment 1/data', file)        
    files.append(tmp)

In [ ]:

def removePuncs(lst):
    
    # This function take list of words as input argument & do the following task
    # Remove Punctuation
    # Remove Stop Words
    # Convert all words in lowercase
    # Return List

    rmvText = ''
    temp = []
    s = ' '
    text = s.join(lst)
    # Remove Punctuation
    nopunc = [char for char in text if char not in string.punctuation]
    for i in nopunc:
        rmvText += i.lower()
    temp = rmvText.split()
    # Remove Stop Words
    temp = ([word for word in temp if word not in stop_words])
    # Retuen list of words
    return temp

In [ ]:
Word_List = []
fileDic = {}
for file in files:
    with open(file,'r',encoding='utf-8', errors='ignore') as f:
        fileDic[file] = removePuncs(f.read().lower().split())
        Word_List.append(removePuncs(f.read().lower().split()))

In [ ]:
# The following code takes all files, replaces new line character with space character and put words of all files in the Words list 
Words = []
for f in files:
    fhand = open(f,'r',encoding='utf-8', errors='ignore')
    file = fhand.read()
    file = file.replace("\n"," ")
    for word in file.split(" "):
        Words.append(word)

In [ ]:
#find count of unique words in all these files and store it in a dictionary
Word_To_Count = {}
for w in Words:
    k = 0
    if not w in Word_To_Count:
        Word_To_Count[w] = k + 1
    else:
        Word_To_Count[w] = Word_To_Count[w] + 1

#""""""OR""""""
# dic = {}
# dic = dict(Counter(Words))
# print(len(dic))


In [ ]:
#print the count of Sweden
Word_To_Count["Sweden"]

53

In [ ]:
#convert all text into lower case then find count of unique words,
Small_Words = [] 
#read each file, split all words and append the lower case word in Small_Words list
for f in files:
    fhand = open(f,'r',encoding='utf-8', errors='ignore')
    file = fhand.read()
    file = file.replace("\n"," ")
    for word in removePuncs(file.split(" ")):
        Small_Words.append(word)

In [ ]:
#find count of unique words after converting them to lowercase in all these files and store it in a dictionary
Small_Word_To_Count = {}
for w in Small_Words:
    k = 0
    if not w in Small_Word_To_Count:
        Small_Word_To_Count[w] = k + 1
    else:
        Small_Word_To_Count[w] = Small_Word_To_Count[w] + 1


In [ ]:
#sort the small words (in Small_Word_To_Count dictionary) by frequencies and store the results in freq.txt file
#Each line of the file should have word seperated by comma with its counts
#Hint: Can you use tuples to sort key value pairs in a list?
fhand = open("Assignment 1/freq.txt",mode='w')
for key, value in sorted(Small_Word_To_Count.items(), key=lambda kv: kv[1], reverse=True):
    fhand.write("%s, %s" % (key, value) + '\n')
fhand.close()

In [ ]:
#sort the words by alphabatical order and store results in words.txt file
#Each line of the file should have word seperated by comma with its counts
fhand = open("Assignment 1/words.txt",mode='w')
for key in sorted(Small_Word_To_Count):
    fhand.write("%s, %s" % (key, Small_Word_To_Count[key]) + '\n')
fhand.close()

In [ ]:
# Sort the list of Small_Words alphabatically and store it in a new list called Vocabulary
# Hint: Explore Set Function
Vocabulary = []
setOfVocabulary = set()
for word in Small_Words:
    setOfVocabulary.add(word)
Vocabulary = list(sorted(setOfVocabulary))

In [ ]:
# Now that the list is sorted alphabatically, 
# We can create dictionaries
Word_to_ID = {}
# Go through the overview.ipynb notebook and convert list items into tuples of indexes and value
# putt all these tuples into Word_to_ID dictionary, Now we can search every words ID
tup = ()
tup = tuple(Vocabulary)
Word_to_ID.update(dict((v,k) for k,v in enumerate(tup)))

In [ ]:
# Now inverse the key value pairs of Word_to_ID Dictionary, so that we can search each word by its ID
ID_to_Word = {}
for k,v in Word_to_ID.items():
  ID_to_Word[v] = k

# An introduction to Sparse Matrix

A sparse matrix is a matrix that is comprised of mostly zero values.
Sparse matrices are distinct from matrices with mostly non-zero values, which are referred to as dense matrices.

"A matrix is sparse if many of its coefficients are zero. The interest in sparsity arises because its exploitation can lead to enormous computational savings and because many large matrix problems that occur in practice are sparse."

— Page 1, Direct Methods for Sparse Matrices, Second Edition, 2017.

Follwing is the example of a Sparse Matrix

<img src="Images/sparse_matrix.png" width="800" height="400">



## Problems with Sparsity


### Space Complexity
Very large matrices require a lot of memory, and some very large matrices that we wish to work with are sparse.
An example of a very large matrix that is too large to be stored in memory is a link matrix that shows the links from one website to another.

In this case, the matrix contained is sparse with many more zero values than data values. The problem with representing these sparse matrices as dense matrices is that memory is required and must be allocated for each 32-bit or even 64-bit zero value in the matrix.

This is clearly a waste of memory resources as those zero values do not contain any information.

### Time Complexity
Assuming a very large sparse matrix can be fit into memory, we will want to perform operations on this matrix.

Simply, if the matrix contains mostly zero-values, i.e. no data, then performing operations across this matrix may take a long time where the bulk of the computation performed will involve adding or multiplying zero values together.

##### Learn More About Sparse Matrrices on Internet. You have to work on sparse matrices in the following tasks

#### Read each document and save it in a Term Document matrix
#### The Size of document vector should be equal to the size of vocabulary


In [ ]:
indptr = [0]
indices = []
data = []
vocabulary = {}
for d in fileDic.values():
    for term in d:
        index = vocabulary.setdefault(term, len(vocabulary))
        indices.append(index)
        data.append(1)
    indptr.append(len(indices))

In [ ]:
CMTDoc = csr_matrix((np.ones(len(data)), indices, indptr), dtype=int).toarray()

In [ ]:
tmp_lst = []
indptr = [0]
indices = []
data = []
vocabulary = {}
for f in range(len(files)):
    fhand = open(files[f],'rt',errors= 'ignore' )
    fdata = fhand.read()
    fdata = fdata.replace("\n"," ")
    for word in removePuncs(fdata.split(" ")):
        index = vocabulary.setdefault(word, len(vocabulary))
        indices.append(index)
        data.append(1)
    indptr.append(len(indices))

In [ ]:
CMTermFre = csr_matrix((data, indices, indptr), dtype=np.intc).toarray()

In [ ]:
#Take Query from the User
Query = input("Enter the search Query: ")
#Convert the query into the query vector, This will be same as we converted each document into document vector
qv = np.zeros((len(Vocabulary)))
for word in Query.split(' '):
    qv[Word_to_ID.get(word.lower())] += 1
# dot product of CMTermFre and Query Vector
dp = CMTermFre.dot(qv)
# Store Results in Dictionaty
resDic = {}
for file in range(len(files)):
    resDic[files[file]] = dp[file]
# sort answer according to highest score
res = list(sorted(resDic.items(), key = lambda k:k[1], reverse = True))
# Top 500 Ranked Document
k = 1
print('\nQuery: ' + Query, '\nResults are:')
for i in res[0:500]:
    print(f'{k}# File Name: {i[0]} , Score : {i[1]/100}.\n')
    k += 1


Enter the search Query: Natural language processing and machine learning

Query: Natural language processing and machine learning 
Results are:
1# File Name: Assignment 1/data/J03-2004.pdf.txt , Score : 186.22.

2# File Name: Assignment 1/data/J06-1005.pdf.txt , Score : 183.74.

3# File Name: Assignment 1/data/J04-3003.pdf.txt , Score : 176.05.

4# File Name: Assignment 1/data/J00-3005.pdf.txt , Score : 164.22.

5# File Name: Assignment 1/data/J00-2002.pdf.txt , Score : 148.93.

6# File Name: Assignment 1/data/D15-1000.pdf.txt , Score : 145.62.

7# File Name: Assignment 1/data/J00-4003.pdf.txt , Score : 138.55.

8# File Name: Assignment 1/data/J03-4003.pdf.txt , Score : 138.17.

9# File Name: Assignment 1/data/J01-2001.pdf.txt , Score : 135.49.

10# File Name: Assignment 1/data/J03-3003.pdf.txt , Score : 134.18.

11# File Name: Assignment 1/data/J05-4005.pdf.txt , Score : 134.0.

12# File Name: Assignment 1/data/J02-3001.pdf.txt , Score : 130.02.

13# File Name: Assignment 1/data/J01-3